In [12]:
import math
import numpy as np
import pandas as pd

In [32]:
# Training and test data files
train_file = '../input/train.csv'
test_file = '../input/test.csv'
model_file = '../output/titanic.model.json'
model_weights_file = '../output/titanic.model.best.hdf5'
pred_file = '../output/gender_submission.csv'

In [33]:
# Load training data
df_train = pd.read_csv(train_file)
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
# Extract survived data as predictions: 0 = Died, 1 = Survived
from keras.utils.np_utils import to_categorical
y_train = to_categorical(df_train["Survived"], 2)
df_train.pop('Survived')
print(y_train.shape)
print(y_train[0:5])

(891, 2)
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [35]:
# Prepare the rest of the data for training
from sklearn.preprocessing import MinMaxScaler

max_name_len = df_train.Name.map(len).max()    

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev', 'Dr', 'Ms', 'Mlle',
            'Col', 'Capt', 'Mme', 'Countess', 'Don', 'Jonkheer']

import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    return np.nan

def prep_data(frame):
    frame = frame.fillna(0)

    # Creating new family_size and fare per person columns 
    frame['Family_Size'] = frame['SibSp'] + frame['Parch'] + 1
    frame['Fare_Per_Person'] = frame['Fare']/frame['Family_Size']

    # Convert Sex and Embarked to number
    frame['Sex'] = pd.Categorical(frame['Sex']).codes
    frame['Embarked'] = pd.Categorical(frame['Embarked']).codes
    
    # Extract title from name
    frame['Title'] = frame['Name'].map(lambda x: substrings_in_string(x, title_list))
    frame['Title'] = pd.Categorical(frame['Title']).codes

    # Convert Name into characters
    frame['Name_Length'] = frame.apply(lambda row: len(row['Name']), axis=1)
#    for i in range(0, max_name_len):
#        col_name = 'Name' + str(i)
#        frame[col_name] = frame['Name'].str[i]
#        frame[col_name] = frame.apply(lambda row: ord(' ') if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Name')

    
    
    # TODO: Ignore Ticket for now
    frame.pop('Ticket')
    
    # Convert Cabin column to whether in cabin
    frame['In_Cabin'] = frame.apply(lambda row: 1 if row['Cabin'] != 0 else 0, axis=1)
    frame.pop('Cabin')
    
    print("Before scaling: ")
    print(frame.head())
    
    # Scale everything except PassengerId
    min_max_scaler = MinMaxScaler()
    col_list = frame.columns.tolist()
    col_list.remove('PassengerId')
    frame = frame[col_list]
    np_scaled = min_max_scaler.fit_transform(frame)
    frame = pd.DataFrame(np_scaled)
    
    print("After scaling: ")
    print(frame.head())

    return frame


In [36]:
# Prepare the rest of the data for training
df_train = prep_data(df_train)
X_train = np.array(df_train)[:,:]
X_train = X_train.astype('float32')
print(X_train.shape)
print(X_train[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0            1       3    1  22.0      1      0   7.2500         3   
1            2       1    0  38.0      1      0  71.2833         1   
2            3       3    0  26.0      0      0   7.9250         3   
3            4       1    0  35.0      1      0  53.1000         3   
4            5       3    1  35.0      0      0   8.0500         3   

   Family_Size  Fare_Per_Person  Title  Name_Length  In_Cabin  
0            2          3.62500     11           23         0  
1            2         35.64165     12           51         1  
2            1          7.92500      8           22         0  
3            2         26.55000     12           44         1  
4            1          8.05000     11           24         0  
After scaling: 
    0    1       2      3    4         5         6    7         8         9   \
0  1.0  1.0  0.2750  0.125  0.0  0.014151  1.000000  0.1  0.007076  0.785714   
1 

In [47]:
# Build a training network

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1], )))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 512)               6656      
_________________________________________________________________
dense_24 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_25 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_28 (Dense)             (None, 256)               65792     
__________

In [48]:
# Train the model
checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=10, verbose=1, mode='auto')
hist = model.fit(X_train, y_train, epochs=1000, batch_size=10, validation_split=0.2, 
                 callbacks=[checkpointer, stopper], 
                 verbose=1, shuffle=True)

Train on 712 samples, validate on 179 samples
Epoch 1/1000
712/712 [==============================] - 2s - loss: 0.5577 - acc: 0.7219 - val_loss: 0.4075 - val_acc: 0.8324
Epoch 2/1000
712/712 [==============================] - 1s - loss: 0.4950 - acc: 0.7949 - val_loss: 0.5569 - val_acc: 0.7654
Epoch 3/1000
712/712 [==============================] - 1s - loss: 0.4851 - acc: 0.7795 - val_loss: 0.4256 - val_acc: 0.8436
Epoch 4/1000
712/712 [==============================] - 1s - loss: 0.4824 - acc: 0.7907 - val_loss: 0.5839 - val_acc: 0.8045
Epoch 5/1000
712/712 [==============================] - 1s - loss: 0.4733 - acc: 0.7992 - val_loss: 0.3874 - val_acc: 0.8324
Epoch 6/1000
712/712 [==============================] - 1s - loss: 0.4851 - acc: 0.8062 - val_loss: 0.3925 - val_acc: 0.8380
Epoch 7/1000
712/712 [==============================] - 1s - loss: 0.4685 - acc: 0.8076 - val_loss: 0.3878 - val_acc: 0.8436
Epoch 8/1000
712/712 [==============================] - 1s - loss: 0.4626 - acc

In [49]:
# Load the weights that yielded the best validation accuracy
model.load_weights(model_weights_file)

# Evaluate the model on the training set
score = model.evaluate(X_train, y_train)
print("\n Training Accuracy:", score[1])

891/891 [==============================] - 0s     

 Training Accuracy: 0.839506173709


In [50]:
# Load test data
df_test_raw = pd.read_csv(test_file)
print(df_test_raw.shape)
df_test_raw.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [51]:
# Prepare the data for testing
df_test = prep_data(df_test_raw)
X_test = np.array(df_test)[:,:]
X_test = X_test.astype('float32')
print(X_test.shape)
print(X_test[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0          892       3    1  34.5      0      0   7.8292         1   
1          893       3    0  47.0      1      0   7.0000         2   
2          894       2    1  62.0      0      0   9.6875         1   
3          895       3    1  27.0      0      0   8.6625         2   
4          896       3    0  22.0      1      1  12.2875         2   

   Family_Size  Fare_Per_Person  Title  Name_Length  In_Cabin  
0            1         7.829200      5           16         0  
1            2         3.500000      6           32         0  
2            1         9.687500      5           25         0  
3            1         8.662500      5           16         0  
4            3         4.095833      6           44         0  
After scaling: 
    0    1         2      3         4         5    6    7         8      9   \
0  1.0  1.0  0.453947  0.000  0.000000  0.015282  0.5  0.0  0.029840  0.625   
1  1

In [52]:
# Predict for test data
y_test = model.predict(X_test)
print(y_test[0])

[ 0.99957401  0.06453325]


In [53]:
# Save predictions
with open(pred_file, 'w') as f:
    f.write('PassengerId,Survived\n')
    for index, y_hat in enumerate(y_test):
        prediction = np.argmax(y_hat)
        f.write(str(df_test_raw.iloc[index]['PassengerId']) + ',' + str(prediction)+'\n')
    f.close()